In [1]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using CUDA
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
	using Flux: @functor
	using ..Transformers: Abstract3DTensor, Container, epsilon, batchedmul, batched_triu!
end
	

In [411]:
#Labeling: vielleicht ändern

begin
	labels = map(string, 1:10)
	startsym = "11"
	endsym = "12"
	unksym = "0"
	labels = [unksym, startsym, endsym, labels...]
	vocab = Vocabulary(labels, unksym)
end

Vocabulary{String}(13, unk=0)

In [412]:
#function for generate training datas 

#nichts ändern
sample_data() = (d = map(string, rand(1:10, 10)); (d,d))

sample_data (generic function with 1 method)

In [413]:
#nichts ändern

sample_data()

(["3", "3", "6", "1", "8", "1", "7", "2", "8", "7"], ["3", "3", "6", "1", "8", "1", "7", "2", "8", "7"])

In [414]:
#function for adding start & end symbol

#nichts ändern
preprocess(x) = [startsym, x..., endsym]

preprocess (generic function with 1 method)

In [415]:
#nichts ändern

begin
    @show sample_ex = preprocess.(sample_data())
    @show encoded_sample_ex = vocab(sample_ex[1]) #use Vocabulary to encode the training data
    end

sample_ex = preprocess.(sample_data()) = (["11", "4", "10", "4", "10", "6", "3", "10", "3", "5", "2", "12"], ["11", "4", "10", "4", "10", "6", "3", "10", "3", "5", "2", "12"])
encoded_sample_ex = vocab(sample_ex[1]) = [2, 7, 13, 7, 13, 9, 6, 13, 6, 8, 5, 3]


12-element Vector{Int64}:
  2
  7
 13
  7
 13
  9
  6
 13
  6
  8
  5
  3

In [416]:
#nichts ändern
sample = preprocess.(sample_data())

(["11", "2", "3", "4", "3", "9", "7", "1", "8", "2", "10", "12"], ["11", "2", "3", "4", "3", "9", "7", "1", "8", "2", "10", "12"])

In [417]:
#nichts ändern
encoded_sample = vocab(sample[1])

12-element Vector{Int64}:
  2
  5
  6
  7
  6
 12
 10
  4
 11
  5
 13
  3

In [418]:
#define a Word embedding layer which turn word index to word vector

#embeding vielleicht ändern
embed = Embed(512, length(vocab)) |> gpu

Embed(512)

In [419]:
#define a position embedding layer metioned above

#embeding vielleicht ändern
pe = PositionEmbedding(512) |> gpu

PositionEmbedding(512)

In [420]:
#wrapper for get embedding

#embeding vielleicht ändern
function embedding(x)
    we = embed(x, inv(sqrt(512)))
    e = we .+ pe(we)
    return e
  end

embedding (generic function with 1 method)

In [421]:
abstract type AbstractAttention end

In [422]:
create_atten_mask1(T::Type, score::AbstractArray, ::Nothing, future) = create_atten_mask1(T, score, fill!(similar(score, size(score,1), size(score, 2), 1), one(T)), future)
function create_atten_mask1(T::Type, score::AbstractArray, _mask::AbstractArray, future::Bool=false)
  #size(mask) == (q, k, n, b)

  # ql, kl = size(mask)
  mask = copy(_mask)

  maskval = convert(T, -1e9)
  !future && batched_triu!(mask, 0)
  mask .= (1 .- mask) .* maskval
  return mask
end

create_atten_mask1 (generic function with 3 methods)

In [423]:
Flux.@nograd create_atten_mask1

In [424]:
struct MultiheadAttention_1{Q<:Dense, K<:Dense, V<:Dense, O<:Dense, DP<:Dropout} <: AbstractAttention
    head::Int
    future::Bool
    iqproj::Q
    ikproj::K
    ivproj::V
    oproj::O
    drop::DP
end

In [425]:
Flux.functor(mh::MultiheadAttention_1) = (mh.iqproj, mh.ikproj, mh.ivproj, mh.oproj), m -> MultiheadAttention_1(mh.head, mh.future, m..., mh.drop)


In [426]:
"""
    MultiheadAttention(head::Int, is::Int, hs::Int, os::Int;
                       future::Bool=true, pdrop = 0.1)
Multihead dot product Attention Layer, `head` is the number of head, 
`is` is the input size, `hs` is the hidden size of input projection layer of each head, 
`os` is the output size. When `future` is `false`, the k-th token can't see tokens at > k. 
`pdrop` is the dropout rate.
"""

"    MultiheadAttention(head::Int, is::Int, hs::Int, os::Int;\n                       future::Bool=true, pdrop = 0.1)\nMultihead dot product Attention Layer, `head` is the number of head, \n`is` is the input size, `hs` is the hidden size of input projection layer of each head, \n`os` is the output size. When `future` is `false`, the k-th token can't see tokens at > k. \n`pdrop` is the dropout rate.\n"

In [427]:
MultiheadAttention_1(head::Int,
                   is::Int,
                   hs::Int,
                   os::Int;
                   future::Bool=true, pdrop = 0.1) = MultiheadAttention_1(head,
                                                                        future,
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(is, hs*head),
                                                                        Dense(hs*head, os),
                                                                        Dropout(pdrop),
                                                                        )


function Base.show(io::IO, mh::MultiheadAttention_1)
    hs = div(size(mh.iqproj.weight)[1], mh.head)
    is = size(mh.iqproj.weight)[end]
    os = size(mh.oproj.weight)[1]

    print(io, "MultiheadAttention(")
    print(io, "head=$(mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "$(is)=>$(os)")

    if Flux.istraining()
        print(io, ", dropout=$(mh.drop.p))")
    else
        print(io, ")")
    end
end

In [428]:
function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: Abstract3DTensor{T},
                         A2 <: Abstract3DTensor{T},
                         A3 <: Abstract3DTensor{T}}
qs = size(query)
ks = size(key)
vs = size(value)

#size(ipq) == (h, q_seq_len, batch)
ipq = @toNd mh.iqproj(query)
ipk = @toNd mh.ikproj(key)
ipv = @toNd mh.ivproj(value)

h = size(ipq, 1)
hs = div(h, mh.head)

#size(ipq) == (hs, q_seq_len, head, batch)
ipq = permutedims(reshape(ipq, hs, mh.head, qs[2], qs[3]), [1, 3, 2, 4])
ipk = permutedims(reshape(ipk, hs, mh.head, ks[2], ks[3]), [1, 3, 2, 4])
ipv = permutedims(reshape(ipv, hs, mh.head, vs[2], vs[3]), [1, 3, 2, 4])

#size(ipq) == (hs, q_seq_len, head * batch)
ipq = reshape(ipq, hs, qs[2], :)
ipk = reshape(ipk, hs, ks[2], :)
ipv = reshape(ipv, hs, vs[2], :)

atten = attention(ipq,ipk,ipv,
mask,
mh.future,
mh.drop)

atten = permutedims(reshape(atten, hs, qs[2], mh.head, qs[3]), [1, 3, 2, 4]) #size(atten) == (hs, head, ql, b)
atten = reshape(atten, h, qs[2], qs[3]) #size(atten) == (h, ql, b)

out = @toNd mh.oproj(atten)
out #size(out) == (h, q_seq_len, batch)
end

function (mh::MultiheadAttention_1)(query::A1,
    key::A2,
    value::A3;
    mask=nothing) where {T,
                         A1 <: AbstractMatrix{T},
                         A2 <: AbstractMatrix{T},
                         A3 <: AbstractMatrix{T}}

# size(query) == (dims, seq_len)
ipq = mh.iqproj(query)
ipk = mh.ikproj(key)
ipv = mh.ivproj(value)

h = size(ipq)[1] #h == hs * head
hs = div(h, mh.head)

#size(hq) == (hs, seq_len, head)
hq = permutedims(reshape(ipq, hs, mh.head, :), [1, 3, 2])
hk = permutedims(reshape(ipk, hs, mh.head, :), [1, 3, 2])
hv = permutedims(reshape(ipv, hs, mh.head, :), [1, 3, 2])

atten = attention(hq, hk, hv,
mask,
mh.future,
mh.drop)

# size(atten) == (head*hs, seq_len)
atten = reshape(permutedims(atten, [1, 3, 2]), h, :)

mh.oproj(atten)
end



In [ ]:
function CausalLinearAttention(queryDimensions)
    feature_map = nelu(queryDimensions)

end

In [4]:
function make_sizes_compatible(Q, K)
    N, L, H, E = Q.shape
    _, S, _, _ = K.shape
        if L == S
            return Q, K
        end
        if L < S
            return Q, K[:, :L, :, :]
        end
        if L > S
            return Q, torch.cat([K, K.new_zeros(N, L-S, H, E)], dim=1)
        end
end

make_sizes_compatible (generic function with 1 method)

In [6]:
function forward(queries, keys, values, attn_mask, query_lengths, key_lengths)
    Q = nelu(queries)
    K = nelu(keys)
    K = K * key_lengths.float_matrix[:, :, None, None]

    Q, K = make_sizes_compatible(Q, K)

    Z = 1/(torch.einsum("nlhi,nlhi->nlh", Q, K.cumsum(1)) + self.eps)  # diese Funktionen und Packages gehen nur mit Torch und CUDA GPU!!

    # Compute the unnormalized result
    V = causal_linear(
        Q,
        K,
        values
    )

    return V * Z[:, :, :, None]
end

forward (generic function with 1 method)

In [429]:
function apply_mask1(score, mask)
    s = size(score)
    ms = size(mask)
    bxn = s[end]
    b = ms[end]
    if bxn == b || b == 1
      return score .+ mask
    else
      return reshape(reshape(score, s[1:end-1]..., :, b) .+
                     reshape(mask, ms[1:end-1]..., 1, b), s)
    end
  end
  
  apply_mask1(score::AbstractArray{T}, ::Nothing, future) where T = future ? score : apply_mask1(score, create_atten_mask1(T, score, nothing, future))
  apply_mask1(score::AbstractArray{T}, mask, future) where T = apply_mask(score, create_atten_mask1(T, score, mask, future))
  

apply_mask1 (generic function with 3 methods)

In [430]:
function attention(query::A1,
    key::A2,
    value::A3,
    mask, future::Bool,
    dropout) where {T,
                    A1 <: Abstract3DTensor{T},
                    A2 <: Abstract3DTensor{T},
                    A3 <: Abstract3DTensor{T}}
#size(query) == (dims, {q,k}_seq_len, batch) == size(key) == size(value)
#size(score) == (k_seq_len, q_seq_len, batch)
dk = size(key, 1)
score = batchedmul(key, query; transA = true)
score = score ./ convert(T, sqrt(dk))

score = apply_mask1(score, mask, future)
score = softmax(score; dims=1)
dropout !== nothing && (score = dropout(score))
batchedmul(value, score) #size(return) == (dims, q_seq_len, batch)
end

attention (generic function with 1 method)

In [431]:
function causal_linear(query, key, value)
    query = query.permute(0,2,1,3).contiguous()
    key = key.permute(0,2,1,3).contiguous()
    value = value.permute(0,2,1,3).contiguous()
    value_new = causal_dot_product(Q, K, V)
    return value_new.permute(0,2,1,3).contiguous()
end

causal_linear (generic function with 1 method)

In [432]:
abstract type AbstractTransformer end

In [433]:
oftf(x, y) = oftype(float(x), y)


nelu(x, α=1) = ifelse(x ≥ 0, float(x)+1, @fastmath oftf(x, α) * (exp(x) - 1)+1)

nelu (generic function with 2 methods)

In [434]:
struct PwFFN{Di<:Dense, Do<:Dense}
    din::Di
    dout::Do
end

@functor PwFFN


"just a wrapper for two dense layer."
PwFFN(size::Int, h::Int, act = elu) = PwFFN(  #relu vielleicht auf elu + 1 oder nur elu ändern
    Dense(size, h, act),
    Dense(h, size)
)

function (pw::PwFFN)(x::AbstractMatrix)
  # size(x) == (dims, seq_len)
  pw.dout(pw.din(x))
end

function (pw::PwFFN)(x::A) where {T, N, A<:AbstractArray{T, N}}
  new_x = reshape(x, size(x, 1), :)
  y = pw(new_x)
  return reshape(y, Base.setindex(size(x), size(y, 1), 1))
end

In [435]:
struct Transformer1{MA<:MultiheadAttention_1, LA<:LayerNorm, P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
    mh::MA
    mhn::LA
    pw::P
    pwn::LP
    drop::DP
end

@functor Transformer1


"""
    Transformer(size::Int, head::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)
    Transformer(size::Int, head::Int, hs::Int, ps::Int;
                future::Bool = true, act = relu, pdrop = 0.1)  

Transformer layer.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
When `future` is `false`, the k-th token can't see the j-th tokens where j > k. `pdrop` is the dropout rate.
"""

########
#ganze Funktion von Transformer ändern
########
function Transformer1(size::Int, head::Int, ps::Int; future::Bool = true, act = elu, pdrop = 0.1)  #relu vielleicht wieder auf elu ändern
    rem(size, head) != 0 && error("size not divisible by head")
    Transformer1(size, head, div(size, head), ps;future=future, act=act, pdrop=pdrop)
end

Transformer1(size::Int, head::Int, hs::Int, ps::Int; future::Bool = true, act = elu, pdrop = 0.1) = Transformer1(
    MultiheadAttention_1(head, size, hs, size; future=future, pdrop=pdrop),
    LayerNorm(size),
    PwFFN(size, ps, act),
    LayerNorm(size),
    Dropout(pdrop),     #braucht man das? vielleicht nicht
)

function (t::Transformer1)(x::A, mask=nothing) where {T, N, A<:AbstractArray{T, N}}
    dropout = t.drop
    a = t.mh(x, x, x; mask=mask)
    a = dropout(a)
    res_a = x + a
    res_a = t.mhn(res_a)
    pwffn = t.pw(res_a)
    pwffn = dropout(pwffn)
    res_pwffn = res_a + pwffn
    res_pwffn = t.pwn(res_pwffn)
    res_pwffn
end

function Base.show(io::IO, t::Transformer1) #zum visualisieren --> nice to have
    hs = div(size(t.mh.iqproj.weight)[1], t.mh.head)
    h, ps = size(t.pw.dout.weight)

    print(io, "Transformer(")
    print(io, "head=$(t.mh.head), ")
    print(io, "head_size=$(hs), ")
    print(io, "pwffn_size=$(ps), ")
    print(io, "size=$(h)")
    if Flux.istraining()
        print(io, ", dropout=$(t.drop.p))")
    else
        print(io, ")")
    end
end


In [436]:
###########
    #auch diese Funktion ändern
    ###############
    struct TransformerDecoder1{MA<:MultiheadAttention_1, LA<:LayerNorm,
        IMA<:MultiheadAttention_1, ILA<:LayerNorm,
        P<:PwFFN, LP<:LayerNorm, DP<:Dropout} <: AbstractTransformer
mh::MA
mhn::LA
imh::IMA
imhn::ILA
pw::P
pwn::LP
drop::DP
end

@functor TransformerDecoder1

"""
TransformerDecoder(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
TransformerDecoder(size::Int, head::Int, hs::Int, ps::Int; act = relu, pdrop = 0.1)

TransformerDecoder layer. Decode the value from a Encoder.

`size` is the input size. if `hs` is not specify, use `div(size, head)` as the hidden size of multi-head attention. 
`ps` is the hidden size & `act` is the activation function of the positionwise feedforward layer. 
`pdrop` is the dropout rate.
"""
function TransformerDecoder1(size::Int, head::Int, ps::Int; act = relu, pdrop = 0.1)
rem(size, head) != 0 && error("size not divisible by head")
TransformerDecoder1(size, head, div(size, head), ps; act=act, pdrop=pdrop)
end

TransformerDecoder1(size::Int, head::Int, hs::Int, ps::Int; act = elu, pdrop = 0.1) = TransformerDecoder1(
MultiheadAttention_1(head, size, hs, size; future=false, pdrop=pdrop),
LayerNorm(size),
MultiheadAttention_1(head, size, hs, size; future=true, pdrop=pdrop), #future = true --> Unterschied zu oben??
LayerNorm(size),
PwFFN(size, ps, act),
LayerNorm(size),
Dropout(pdrop),
)

function (td::TransformerDecoder1)(x::AbstractArray{T,N}, m, mask=nothing) where {T,N}
dropout = td.drop
a = td.mh(x,x,x)
a = dropout(a)
res_a = x + a
res_a = td.mhn(res_a)

ia = td.imh(res_a, m, m, mask=mask)
ia = dropout(ia)
res_ia = res_a + ia
res_ia = td.imhn(res_ia)

pwffn = td.pw(res_ia)
pwffn = dropout(pwffn)
res_pwffn = res_ia + pwffn
res_pwffn = td.pwn(res_pwffn)
res_pwffn
end

function Base.show(io::IO, td::TransformerDecoder1)
hs = div(size(td.imh.iqproj.weight)[1], td.imh.head)
h, ps = size(td.pw.dout.weight)

print(io, "TransformerDecoder(")
print(io, "head=$(td.mh.head), ")
print(io, "head_size=$(hs), ")
print(io, "pwffn_size=$(ps), ")
print(io, "size=$(h)")
if Flux.istraining()
print(io, ", dropout=$(td.drop.p))")
else
print(io, ")")
end
end

In [437]:
#define 2 layer of transformer

#layer auf jeden Fall ändern
encode_t1 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [438]:
encode_t2 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [439]:
encode_t3 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [440]:
encode_t4 = Transformer1(512, 8, 64, 2048) |> gpu

Transformer(head=8, head_size=64, pwffn_size=2048, size=512)

In [441]:
#define 2 layer of transformer decoder

#layer auf jeden Fall ändern
decode_t1 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [442]:
decode_t2 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [443]:
decode_t3 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [444]:
decode_t4 = TransformerDecoder1(512, 8, 64, 2048) |> gpu

TransformerDecoder(head=8, head_size=64, pwffn_size=2048, size=512)

In [445]:
#define the layer to get the final output probabilities

#ÄNDERN
linear = Positionwise(Dense(512, length(vocab)), logsoftmax) |> gpu

Positionwise(Dense(512, 13), logsoftmax)

In [446]:
#neue Layer einfügen
function encoder_forward(x)
    e = embedding(x)
    t1 = encode_t1(e)
    t2 = encode_t2(t1)
    t3 = encode_t2(t2)
    t4 = encode_t2(t3)
    return t2
  end

encoder_forward (generic function with 1 method)

In [447]:
#neue Layer einfügen
function decoder_forward(x, m)
    e = embedding(x)
    t1 = decode_t1(e, m)
    t2 = decode_t2(t1, m)
    t3 = decode_t3(t2, m)
    t4 = decode_t4(t3, m)
    p = linear(t4)
    return p
  end

decoder_forward (generic function with 1 method)

In [448]:
#bleibt so
enc = encoder_forward(encoded_sample)

512×12 Matrix{Float32}:
 -1.13408     -1.84392   -2.32267     …  -1.64319    -1.0968     -0.520561
 -1.08517     -1.09036   -1.56736        -1.85067    -2.38069    -2.34225
  0.831087     0.302583   0.0664692      -0.060868    0.324584    0.749434
  0.848626     0.897475   0.506241        0.0141686  -0.394741   -0.433558
  0.00865521  -0.487076  -0.876659       -1.46744    -1.19967    -0.635452
 -0.794482    -0.379881  -0.706728    …  -0.990828   -1.63664    -2.15277
 -1.38286     -1.79868   -2.20306        -1.89379    -1.91283    -1.45226
  0.334699     0.482382   0.3129          0.588992   -0.0460867  -0.523754
  0.0969949   -0.413367  -0.763434       -1.34569    -1.56875    -1.212
  0.623114     0.414014  -0.15226        -0.0594968  -0.455226   -1.12402
  ⋮                                   ⋱               ⋮          
 -0.588119    -0.585887  -0.739805       -0.835556   -0.710021   -0.569935
 -2.18158     -2.07123   -1.91992        -1.90608    -1.81531    -1.84507
 -1.09231     -1.0

In [449]:
#bleibt so
probs = decoder_forward(encoded_sample, enc)

13×12 Matrix{Float32}:
 -3.33083  -3.37172  -3.37812  -3.40815  …  -3.67683  -3.58109  -3.50331
 -4.31708  -4.26506  -4.19566  -4.16887     -4.01353  -3.97492  -3.96483
 -3.38153  -3.38715  -3.43217  -3.45597     -3.43957  -3.42064  -3.41785
 -2.56638  -2.58689  -2.60332  -2.61612     -2.48629  -2.49056  -2.48591
 -3.96521  -3.97472  -3.97081  -4.01278     -4.14631  -4.16042  -4.16306
 -1.99491  -1.91175  -1.83148  -1.77007  …  -1.71046  -1.71809  -1.76508
 -1.72637  -1.72586  -1.7042   -1.67745     -1.54591  -1.52572  -1.52684
 -2.16321  -2.16004  -2.13376  -2.16662     -2.47786  -2.42628  -2.39656
 -4.40395  -4.45754  -4.53692  -4.62177     -4.37682  -4.41079  -4.45209
 -1.35455  -1.37756  -1.43772  -1.46047     -1.54932  -1.58851  -1.56803
 -2.76795  -2.81047  -2.82594  -2.84114  …  -2.76693  -2.8033   -2.8192
 -3.80526  -3.79245  -3.76785  -3.78733     -3.6266   -3.60324  -3.56655
 -3.30018  -3.32112  -3.36363  -3.38169     -3.40323  -3.41862  -3.44518

In [450]:
#hier vielleicht etwas ändern --> herausfinden, was diese funktion macht
function smooth(et)
    sm = fill!(similar(et, Float32), 1e-6/size(embed, 2))
    p = sm .* (1 .+ -et)
    label = p .+ et .* (1 - convert(Float32, 1e-6))
    label
end

smooth (generic function with 1 method)

In [451]:
Flux.@nograd smooth

In [452]:
#define loss function

#ÄNDERN
function loss(x, y)
    label = onehot(vocab, y) #turn the index to one-hot encoding
    label = smooth(label) #perform label smoothing
    enc = encoder_forward(x)
    probs = decoder_forward(y, enc)
    l = logkldivergence(label[:, 2:end, :], probs[:, 1:end-1, :])  #erstmal so lassen
    return l
  end

loss (generic function with 1 method)

In [453]:
#collect all the parameters

#Layer neu einfügen
ps = params(embed, pe, encode_t1, encode_t2, encode_t3, encode_t4, decode_t1, decode_t2, decode_t3, decode_t4, linear)

Params([Float32[0.44517747 -1.2352791 … -0.5580842 0.61990374; -0.86038107 0.9852316 … 0.8646372 -1.6544943; … ; -0.4268172 -0.24769828 … -1.0369782 -0.66909796; -0.59597033 0.7646327 … 1.0684395 -0.24535699], Float32[0.5403023 -0.41614684 … 0.4000682 0.9873536; 0.8218562 0.9364147 … 0.37902638 0.97646356; … ; 1.0 1.0 … 0.9943822 0.99437124; 0.0001 0.0002 … 0.10212166 0.10222114], Float32[-0.041287433 -0.048739977 … 0.040234454 0.030739756; -0.046966214 0.054721765 … -0.017682707 -0.028461229; … ; 0.017833525 -0.022631519 … -0.024375683 -0.03949702; 0.07064739 0.021828623 … 0.076225206 -0.032440905], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.04466606 0.023011195 … -0.010626206 0.029662123; -0.020044418 0.058562506 … -0.0024756473 -0.038418476; … ; -0.030274872 -0.008325687 … -0.038550917 0.061701123; 0.032254297 0.06236364 … 0.031859256 -0.060116887], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [454]:
opt = ADAM(1e-4) #bleibt

ADAM(0.0001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [455]:
#define training loop

#ÄNDERN siehe Paper Kapitel 4.1
function train!()
    @info "start training"
    for i = 1:1000
      data = batched([sample_data() for i = 1:32]) #create 32 random sample and batched
      x, y = preprocess.(data[1]), preprocess.(data[2])
      x, y = vocab(x), vocab(y) #encode the data
      x, y = todevice(x, y) #move to gpu
      grad = gradient(()->loss(x, y), ps)
      if i % 8 == 0
          l = loss(x, y)
          println("loss = $l")
      end
      update!(opt, ps, grad)
    end
  end

train! (generic function with 1 method)

In [456]:
train!()

┌ Info: start training
└ @ Main /Users/johannes/Documents/GitHub/DM2022-LinearTransformers/test/Duplicat_Exercise11.ipynb:5


loss = 105.71132


loss = 82.529884


loss = 81.73267


loss = 79.40291


...

loss = 0.010820839


loss = 0.017511562


loss = 0.0071364627


loss = 0.0067240065


loss = 0.0069205435


In [457]:
#erst einmal so lassen, vielleicht einzelne Änderungen
function translate(x)
    ix = todevice(vocab(preprocess(x)))
    seq = [startsym]

    enc = encoder_forward(ix)

    len = length(ix)
    for i = 1:2len
        trg = todevice(vocab(seq))
        dec = decoder_forward(trg, enc)
        #move back to gpu due to argmax wrong result on CuArrays
        ntok = onecold(collect(dec), labels)
        push!(seq, ntok[end])
        ntok[end] == endsym && break
    end
  seq[2:end-1]
end

translate (generic function with 1 method)

In [458]:
translate(map(string, [5,5,6,6,1,12,3,4,6]))

10-element Vector{String}:
 "5"
 "5"
 "6"
 "6"
 "1"
 "1"
 "3"
 "4"
 "6"
 "6"